In [2]:
# data analysis and wrangling
import pandas as pd
import numpy as np
import pandas_profiling as pdprof

#data modeling
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC

# visualization
import seaborn as sns
import matplotlib.pyplot as plt




In [9]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor

In [26]:
from sklearn.neighbors import KNeighborsClassifier

In [3]:
train = pd.read_csv("Data/trainingData.csv")
test = pd.read_csv("Data/validationData.csv")

In [ ]:
pdprof.ProfileReport(train[[ 'LONGITUDE',
 'LATITUDE',
 'FLOOR',
 'BUILDINGID',
 'SPACEID',
 'RELATIVEPOSITION',
 'USERID',
 'PHONEID',
 'TIMESTAMP']])

In [11]:
train.dtypes
train.columns
train.iloc[:,522:528] = train.iloc[:,522:528].apply(lambda x: x.astype('category'))
test.iloc[:,522:528] = test.iloc[:,522:528].apply(lambda x: x.astype('category'))


In [4]:
#base_model_building = train.filter(regex='^WAP').assign(BUILDINGID = BUILDINGID df: train.BUILDINGID)
base_model_building = train.filter(regex=['^WAP', 'BUILDINGID'])
testsubset = train.filter(regex=['^WAP', 'BUILDINGID'])
base_model_building['BUILDINGID'] = train['BUILDINGID']
testsubset

TypeError: unhashable type: 'list'

In [13]:
rfc = RandomForestClassifier()
rfc.fit(base_model_building.filter(regex='^WAP'), base_model_building['BUILDINGID'])
BUILDINGID_pred = rfc.predict(test.filter(regex='^WAP'))
confusion_matrix(test.BUILDINGID,BUILDINGID_pred)

C:\Users\Alfonso\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


array([[535,   1,   0],
       [  1, 306,   0],
       [  0,   0, 268]], dtype=int64)

In [22]:
b1 = train[train.BUILDINGID == 1]
b1_floor_y= b1.FLOOR
b1_floor_x= b1.filter(regex='^WAP')
b1_test =  test[test.BUILDINGID == 1]
b1_floor_test_x = b1_test.filter(regex='^WAP')
b1_floor_test_y= b1_test.FLOOR

In [51]:
rfc_b1_floor = RandomForestClassifier()
rfc_b1_floor.fit(b1_floor_x,b1_floor_y)
rfc_b1_floor_pred = rfc_b1_floor.predict(b1_test.filter(regex='^WAP'))
print(confusion_matrix(rfc_b1_floor_pred,b1_floor_test_y))
print(accuracy_score(rfc_b1_floor_pred,b1_floor_test_y))
print(cohen_kappa_score(rfc_b1_floor_pred,b1_floor_test_y))

[[23  9  0  0]
 [ 0 83  2  0]
 [ 6 48 77  6]
 [ 1  3  8 41]]
0.7296416938110749
0.6181134224566873


C:\Users\Alfonso\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [54]:
neighbors_settings = range(1,12,2)
for k in neighbors_settings:
    classifier = KNeighborsClassifier(n_neighbors=k)
    classifier.fit(b1_floor_x,b1_floor_y)
    y_pred = classifier.predict(b1_floor_test_x)
    print(k)
    print(confusion_matrix(y_pred,b1_floor_test_y))
    print(accuracy_score(y_pred,b1_floor_test_y))
    print(cohen_kappa_score(y_pred,b1_floor_test_y))


1
[[21 11  0  0]
 [ 4 83  1  0]
 [ 5 44 56  2]
 [ 0  5 30 45]]
0.6677524429967426
0.5370217044177656
3
[[23 15  0  0]
 [ 2 80  1  0]
 [ 3 42 58  3]
 [ 2  6 28 44]]
0.6677524429967426
0.5412405871839199
5
[[22 13  0  0]
 [ 2 81  2  0]
 [ 3 44 53  3]
 [ 3  5 32 44]]
0.6514657980456026
0.5178836134145447
7
[[22 14  0  0]
 [ 2 78  2  0]
 [ 3 46 55  3]
 [ 3  5 30 44]]
0.6482084690553745
0.5144112478031635
9
[[22 15  0  0]
 [ 2 77  2  0]
 [ 3 44 58  2]
 [ 3  7 27 45]]
0.6579804560260586
0.5286798356556959
11
[[21 13  0  0]
 [ 2 79  2  0]
 [ 3 44 57  2]
 [ 4  7 28 45]]
0.6579804560260586
0.5272767267927848


In [47]:
classifier5 = KNeighborsClassifier(n_neighbors=5)
classifier5.fit(b1_floor_x,b1_floor_y)
y_pred = classifier.predict(b1_floor_test_x)
print(confusion_matrix(y_pred,b1_floor_test_y),"K=5")


[[22 14  0  0]
 [ 2 82  2  0]
 [ 3 40 58  2]
 [ 3  7 27 45]] K=5


In [58]:
svclassifier = SVC(kernel='linear')
svclassifier.fit(b1_floor_x,b1_floor_y)
svclassifier_pred = svclassifier.predict(b1_floor_test_x)
print(confusion_matrix(svclassifier_pred,b1_floor_test_y))
print(accuracy_score(svclassifier_pred,b1_floor_test_y))
print(cohen_kappa_score(svclassifier_pred,b1_floor_test_y))

[[20 22  0  0]
 [ 7 69  2  0]
 [ 3 49 61 10]
 [ 0  3 24 37]]
0.6091205211726385
0.4592372955993307


In [59]:
#NN with keras
from keras.models import Sequential
from keras.layers import Dense

ModuleNotFoundError: No module named 'keras'

In [ ]:
from sklearn.ensemble import RandomForestClassifier
#***************************************Random Forest
model2 = RandomForestClassifier(n_estimators=200, random_state=3, max_depth=18,
                                        min_weight_fraction_leaf=0.0002)

In [ ]:
from scipy.stats import randint as sp_randint
param_dist = {"max_depth": sp_randint(3, 10),
              "max_features": sp_randint(7, 15),
              "min_samples_split": sp_randint(2, 11),
              "min_samples_leaf": sp_randint(1, 11),
              "bootstrap": [True, False]
             }

In [ ]:
model2.fit(base_model_building.filter(regex='^WAP'), base_model_building['BUILDINGID'])
print("Model2 trained")
preds2 = model2.predict(test.filter(regex='^WAP'))

In [4]:
confusion_matrix(test.BUILDINGID,preds2)

NameError: name 'test' is not defined